In [21]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [22]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [23]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    #Initializing Values and policy.
    V = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    
    # Implement!
    while True:
        delta = 0
        
        #Going through all of the states, one by one.
        for s in range(env.nS):
            

            v = 0
            expected_values = np.zeros(env.nA)
        #For updating the value, we do one step-look-ahead    
            for a in range(env.nA):
                #We keep an array of expected returns from all of the actions possible 
                for prob, next_state, reward, done in env.P[s][a]:
                    expected_values[a] += prob*(reward+discount_factor*V[next_state])
                
        #Choosing value as the max of all the possible returns we can get from the actions possible       
            v = np.max(expected_values) 
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    #for policy, just act greedily w.r.t. this value function.
    for s in range(env.nS):
    #To act greeddily, we do one step-look-ahead, again.
        expected_values = np.zeros(env.nA)
        for a in range(env.nA):
            #We keep an array of expected returns from all of the actions possible 
            for prob, next_state, reward, done in env.P[s][a]:
                expected_values[a] += prob*(reward+discount_factor*V[next_state])
    #Creating new policy as the action for each state that maximizes the expected return 
        new_action = np.argmax(expected_values)
    #Updating the old policy
        policy[s] = np.eye(env.nA)[new_action]
    return policy, V

In [24]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [26]:
%timeit value_iteration(env)

816 µs ± 59.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


As you can see, the convergence of the Value Iteration Algorithm is order times better than Policy Iteration.

In [25]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)